# Object Detection from Images



In [ ]:
import numpy as np 
import tensorflow as tf
import pandas as pd
import tensorflow.keras
from tensorflow.keras.datasets import cifar10

### Loading the dataset

In [ ]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
print(x_test.shape)
print(y_test.shape)

In [ ]:
x_train[0]

In [ ]:
y_train

#### Re-scaling so that all the pixel values lie within 0 to 1

In [ ]:
x_train = x_train.astype('float32')/255
x_test = x_test.astype('float32')/255

In [ ]:
x_train[0]

In [ ]:
no_of_classes = len(np.unique(y_train))
no_of_classes

### Transforming it into One-hot Encoded form

In [3]:
y_train = tensorflow.keras.utils.to_categorical(y_train,no_of_classes)
y_test = tensorflow.keras.utils.to_categorical(y_test,no_of_classes)
y_test

### Dividing our training set into 2 sets - train and validation.

In [ ]:
x_train,x_valid = x_train[5000:],x_train[:5000]
y_train,y_valid = y_train[5000:],y_train[:5000]
print(x_train.shape)
print(y_train.shape)

In [ ]:
print(x_valid.shape)
print(y_valid.shape)

In [ ]:
#Showing the pictures of First 20 images from training data set
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(20,20))
for i in range(25):
    ax = fig.add_subplot(5,5,i+1,xticks=[],yticks=[])
    ax.imshow(np.squeeze(x_train[i]))
    

## Creating Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten,Dropout,MaxPooling2D

model = Sequential()
model.add(Conv2D(filters=16, kernel_size = 2, padding = 'same',activation = 'relu',input_shape=(32,32,3)))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=32, kernel_size = 2, padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size = 2, padding = 'same',activation = 'relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(10,activation='softmax'))

model.summary()

### Compiling model

In [ ]:
model.compile(optimizer = 'adam', loss ='categorical_crossentropy',metrics=['accuracy'])
print('Model compiled!')

### Start training the model - 3..... 2...... 1...... Go

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint(filepath = 'best_model_.h5',save_best_only = True,verbose=1) #Save the best model in the file path

history = model.fit(x_train,y_train,batch_size=32, epochs = 15,
          validation_data=(x_valid,y_valid),
          callbacks=[checkpoint],
          verbose=2, 
          shuffle=True)

### Checking the accuracy score of the best model on our test dataset

In [8]:
model.load_weights('Best_Basic_Model.h5')
score = model.evaluate(x_test,y_test,verbose=0)
score[1]

#Accuracy ranging from 73% to 78%

### Visualizing the accuracy and loss over the epochs.


In [ ]:
plt.figure(1)  
   
#summarizing history for accuracy  

plt.subplot(211)  
plt.plot(history.history['accuracy'])  
plt.plot(history.history['val_accuracy'])  
plt.title('Model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
   

#summarizing history for loss  
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()  

# --------------------------------------------------------------------------------------------------------------

# Pre-trained Model using VGG-16

In [ ]:
import numpy as np
import pandas as pd
from numpy.random import rand
pd.options.display.max_colwidth = 600
from sklearn import preprocessing
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.model_selection import train_test_split
# # Matplot Imports
# import matplotlib.pyplot as plt
# params = {'legend.fontsize': 'x-large',
#           'figure.figsize': (15, 5),
#           'axes.labelsize': 'x-large',
#           'axes.titlesize':'x-large',
#           'xtick.labelsize':'x-large',
#           'ytick.labelsize':'x-large'}

# plt.rcParams.update(params)
# %matplotlib inline

# # pandas display data frames as tables
# from IPython.display import display, HTML

# import warnings
# warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
from keras import callbacks
from keras import optimizers
from keras.datasets import cifar10
from keras.engine import Model
from keras.applications import vgg16 as vgg
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D,BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
BATCH_SIZE = 32
EPOCHS = 100
NUM_CLASSES = 10
LEARNING_RATE = 1e-4
MOMENTUM = 0.9

Since I am using VGG16 as a feature extractor, the minimum size of an image it takes is 48x48.

In [ ]:
from skimage.transform import resize
from skimage import data
x = data.camera()
X_train = np.array([resize(x, (48, 48)) for x in x_train])
X_val = np.array([resize(x, (48, 48)) for x in x_valid])
X_test = np.array([resize(x, (48, 48)) for x in x_test])

## Preparing the Model

* Loading VGG16 without the top classification layer
* Preparing a custom classifier
* Stacking both models on top of each other

In [ ]:
vgg_model = vgg.VGG16(weights='imagenet', 
                       include_top=False, 
                       input_shape=(48, 48, 3))

 Extracting the last layer from third block of VGG16 model

In [ ]:
last = vgg_model.get_layer('block3_pool').output

 Adding Classification layers on top of it

In [ ]:
x = GlobalAveragePooling2D()(last)
x= BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.4)(x)
pred = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(vgg_model.input, pred)

Only training the custom classifier, we freeze the layers of VGG16

In [ ]:
for layer in vgg_model.layers:
     layer.trainable = False

## Compiling VGG16 model

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=LEARNING_RATE),
              metrics=['accuracy'])

print('Model compiled!')

In [ ]:
model.summary()

##  Data Augmentation 

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True)

In [ ]:
# For Training Data

train_datagen.fit(X_train)
train_generator = train_datagen.flow(X_train,
                                     y_train, 
                                     batch_size=BATCH_SIZE)

In [ ]:
# For Validation Data

val_datagen = ImageDataGenerator(rescale=1. / 255,
    horizontal_flip=False)

val_datagen.fit(X_val)
val_generator = val_datagen.flow(X_val,
                                 y_valid,
                                 batch_size=BATCH_SIZE)

In [ ]:
train_steps_per_epoch = X_train.shape[0] // BATCH_SIZE
val_steps_per_epoch = X_val.shape[0] // BATCH_SIZE


### Start training the VGG16 model - 3..... 2...... 1...... Go

In [ ]:
from keras.callbacks import ModelCheckpoint

checkpoint1 = ModelCheckpoint(filepath = 'Best_Model_Pretrained_.h5',save_best_only = True,verbose=1)

history1 = model.fit(train_generator,
                              steps_per_epoch=train_steps_per_epoch,
                              validation_data=val_generator,
                              validation_steps=val_steps_per_epoch,
                              epochs=EPOCHS,
                              callbacks=[checkpoint1],
                              verbose=1)

### Checking the accuracy score of the best model on our test dataset

In [ ]:
model.load_weights('Best_Model_Pretrained_.h5')
score1 = model.evaluate(X_test,y_test,verbose=0)
score1[1]

# Getting accuracy of around 83% on Epoch=40

### Visualizing the accuracy and loss over the epochs.


In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(35, 7))
t = f.suptitle('Deep Neural Net Performance', fontsize=12)
f.subplots_adjust(top=0.85, wspace=0.3)

epochs = list(range(1,EPOCHS+1))
ax1.plot(epochs, history1.history['accuracy'], label='Train Accuracy')
ax1.plot(epochs, history1.history['val_accuracy'], label='Validation Accuracy')
ax1.set_xticks(epochs)
ax1.set_ylabel('Accuracy Value')
ax1.set_xlabel('Epoch')
ax1.set_title('Accuracy')
l1 = ax1.legend(loc="best")

ax2.plot(epochs, history1.history['loss'], label='Train Loss')
ax2.plot(epochs, history1.history['val_loss'], label='Validation Loss')
ax2.set_xticks(epochs)
ax2.set_ylabel('Loss Value')
ax2.set_xlabel('Epoch')
ax2.set_title('Loss')
l2 = ax2.legend(loc="best")